In [3]:
from gbdxtools import Interface
from gbdxtools import CatalogImage
from gbdxtools.task import env
import json

gbdx = Interface()

In [4]:
catalog_id = env.inputs.get('catalog_id', '10500100012D5600')
image = CatalogImage(catalog_id)
imgBounds = image.bounds
print(imgBounds)

(-77.67405014, 0.6903576118900467, -77.51874176773985, 1.02782883)


In [5]:
metadata = image.ipe.metadata['image']
#print(json.dumps(metadata, indent=4))

In [5]:
data_url = 's3://receiving-dgcs-tdgplatform-com/057792790010_01_003'

In [6]:
ao_task = gbdx.Task('AOP_Strip_Processor',
                    data=data_url,
                    bands='MS',
                    dra_mode='BaseLayerMatch')
ao_task.impersonation_allowed = True

In [8]:
aoiBounds = [-77.669545, 0.809357, -77.515784, 0.959268]

print('image bounds', imgBounds)
print('chose bounds', aoiBounds)

bounds = [max(imgBounds[0], aoiBounds[0]), max(imgBounds[1], aoiBounds[1]),
                min(imgBounds[2], aoiBounds[2]), min(imgBounds[3], aoiBounds[3])]

print('fixed bounds', bounds)

('image bounds', (-77.67405014, 0.6903576118900467, -77.51874176773985, 1.02782883))
('chose bounds', [-77.669545, 0.809357, -77.515784, 0.959268])
('fixed bounds', [-77.669545, 0.809357, -77.51874176773985, 0.959268])


In [9]:
# problem: ship value may not be set
clip_task = gbdx.Task('gdalcrop',                      
                     #image='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/lt_ao_bs_match1/',
                     image=ao_task.outputs.data.value,
                     crop='{} {} {} {}'.format(bounds[0], bounds[1], bounds[2], bounds[3]),
                     ship="false")

In [10]:
tile_task = gbdx.Task('splitter-gbdx:0.0.5',
                     data_in=clip_task.outputs.cropped.value)

In [11]:
fe_task = gbdx.Task('fe-gbdx:0.0.9',
                   data_in=tile_task.outputs.data_out.value)

In [13]:
clustervis_task = gbdx.Task('clustervis-gbdx:0.0.14',
                           data_in=tile_task.outputs.data_out.value,
                           features_in=fe_task.outputs.data_out.value,
                           #data_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/tiles2',
                           #features_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/fe-out2',
                           train='false')

In [14]:
#tasks = [ao_task]
#tasks2 = [clip_task]
#tasks = [ao_task, clip_task]
#tasks = [clip_task, tile_task]
#tasks = [fe_task]
#tasks = [clustervis_task]
#tasks = [clip_task, tile_task, fe_task, clustervis_task]
tasks = [ao_task, clip_task, tile_task, fe_task, clustervis_task]
workflow = gbdx.Workflow(tasks)

In [15]:
workflow.savedata(ao_task.outputs.data, location='colombia-2-acomp')
workflow.savedata(clip_task.outputs.cropped, location='colombia-2-clip')
workflow.savedata(tile_task.outputs.data_out, location='colombia-2-tiles')
workflow.savedata(fe_task.outputs.data_out, location='colombia-2-fe')
workflow.savedata(clustervis_task.outputs.data_out, location='colombia-2-out')

In [16]:
workflow.execute()

u'4837083965250102695'

In [19]:
print workflow.id
print workflow.status
#print(json.dumps(workflow.events[len(workflow.events)-1], indent=4))
#print(json.dumps(workflow.events, indent=4))
for e in workflow.events:
    print("    {} {}".format(e['task'], e['event']))
    #print(e)
print(json.dumps(workflow.stdout, indent=4))
print(json.dumps(workflow.stderr, indent=4))

4837083965250102695
{u'state': u'running', u'event': u'started'}
    AOP_Strip_Processor_c8e3821d submitted
    gdalcrop_9ecd10e8 submitted
    splitter-gbdx_0.0.5_a00b6d7c submitted
    fe-gbdx_0.0.9_f2283492 submitted
    clustervis-gbdx_0.0.14_aa039911 submitted
    AOP_Strip_Processor_c8e3821d scheduled
    AOP_Strip_Processor_c8e3821d started
[
    {
        "stdout": "Unable to retrieve stdout.  Unable to locate item with workflow ID '4837083965250102695' and task ID '4837083965218726873'", 
        "id": "4837083965218726873", 
        "name": "AOP_Strip_Processor_c8e3821d", 
        "taskType": "AOP_Strip_Processor:0.0.4"
    }, 
    {
        "stdout": "Unable to retrieve stdout.  Unable to locate item with workflow ID '4837083965250102695' and task ID '4837083965234695450'", 
        "id": "4837083965234695450", 
        "name": "clustervis-gbdx_0.0.14_aa039911", 
        "taskType": "clustervis-gbdx:0.0.14"
    }, 
    {
        "stdout": "Unable to retrieve stdout.  Unable 

In [ ]:
workflow.cancel()